In [5]:
import pandas as pd
from datetime import datetime
import dask as dd
csv_file_path = 'dane_1.csv'
df_log_csv = pd.read_csv(csv_file_path)

lines_csv = df_log_csv.iloc[:, 0].tolist()


def parse(inp: str):
    record = {}
    date_start = inp.find('[') + 1
    date_end = inp.find(']')
    date_s = slice(date_start, date_end)

    level_start = inp.find('[', date_end) + 1
    level_end = inp.find(']', level_start)
    level_s = slice(level_start, level_end)

    client_start = inp.find('[', level_end)
    client_end = inp.find(']', client_start)

    record["date"] = inp[date_s]
    record["level"] = inp[level_s]
    record["client"] = "" if client_start == - \
        1 else inp[client_start + 8: client_end]
    record["message"] = inp[client_end +
                            2:] if record["client"] else inp[level_end + 2:]
    return record


def convert_date(rec):
    rec["date"] = datetime.strptime(rec["date"], "%a %b %d %H:%M:%S %Y")
    return rec


output_csv = []
for line in lines_csv:
    record = parse(line)
    record = convert_date(record)
    output_csv.append(list(record.values()))

df_parsed_logs_csv = pd.DataFrame(
    output_csv, columns=["date", "level", "client", "message"])

print(df_parsed_logs_csv.head())

                 date   level client  \
0 2005-12-04 04:47:44   error          
1 2005-12-04 04:51:08  notice          
2 2005-12-04 04:51:09  notice          
3 2005-12-04 04:51:09  notice          
4 2005-12-04 04:51:14  notice          

                                             message  
0            mod_jk child workerEnv in error state 6  
1  jk2_init() Found child 6725 in scoreboard slot 10  
2   jk2_init() Found child 6726 in scoreboard slot 8  
3   jk2_init() Found child 6728 in scoreboard slot 6  
4  workerEnv.init() ok /etc/httpd/conf/workers2.p...  


In [13]:
import dask.bag as db
import json
import os
import dask as dd
DATAPATH = '/M'
os.makedirs(DATAPATH, exist_ok=True)

b = dd.datasets.make_people(npartitions=100, records_per_partition=10000)


def is_expired(record):
    month, year = map(int, record["credit-card"]["expiration-date"].split('/'))
    return year < 2024 or (year == 2024 and month < 10)


expired = b.filter(is_expired)

expired.repartition(10).map(json.dumps).to_textfiles(
    os.path.join(DATAPATH, 'expired_*.json'))

print("Zadanie 2 zakończone: Rekordy z wygasłymi kartami kredytowymi zapisane do plików JSON.")

Zadanie 2 zakończone: Rekordy z wygasłymi kartami kredytowymi zapisane do plików JSON.


In [14]:
adults = b.filter(lambda record: record['age'] >= 18)

df_adults = adults.to_dataframe()

df_adults.to_parquet(os.path.join(DATAPATH, 'adults.parquet'),
                     write_index=False, engine='pyarrow')

print("Zadanie 3 zakończone: Dane osób dorosłych zapisane do pliku parquet.")

Zadanie 3 zakończone: Dane osób dorosłych zapisane do pliku parquet.
